In [6]:
%matplotlib inline
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pysentani


In [7]:
survey = pd.read_excel('../data-clean/sentani-merged-cleaned-2015-06-10.xlsx',na_values=[''])
print('number of entries =', len(survey))
print('number of columns =', len(survey.columns))

number of entries = 1184
number of columns = 274


In [12]:
pysentani.similar_columns(survey,'genset')

power_supply/private_genset
private_genset_year
private_genset_size_w_or_VA
private_genset_size_w
private_genset_size_volts
private_genset_size_amps
private_genset_price
power_supply_working/private_genset
genset_expenditure


In [29]:
household_survey = survey[survey['demand_point'] == 'household']
hh_series = pd.value_counts(household_survey['village_name'])
print(hh_series)

Ayapo           113
Sosiri           94
Flafow           68
Puai             67
Babrongko        59
Yakonde          57
Simporo          54
Yoboi            46
Donday           43
Yokiwa           38
Abar             38
Atamali          34
Kampung_Baru     33
Ebunfauw         30
Kwadeware        28
Yobeh            28
Yoka             27
Kalio            27
Asei             25
Kheleubulow      23
Evale            22
Hobong           19
Kensio           18
Khageuw          16
Ajau             14
Pantai_Yahim     13
Khamayakha       12
Obolyo           10
Burawai           5
dtype: int64


In [13]:
household_survey[['power_supply/private_genset','power_supply_working/private_genset']]

,power_supply/private_genset,power_supply_working/private_genset
0,1,1
1,NaN,NaN
2,0,0
3,NaN,NaN
4,1,1
5,1,1
6,1,1
7,0,0
8,NaN,NaN
9,1,1


In [15]:
pd.crosstab(household_survey['power_supply/private_genset'],household_survey['power_supply_working/private_genset'])

power_supply_working/private_genset,0.0,1.0
power_supply/private_genset,,
0,735,19
1,9,210


In [51]:
go_df = household_survey.groupby(['village_name'])[['power_supply_working/private_genset']].agg(['count'])
#type(go_df['power_supply_working/private_genset'])
go_series = go_df['power_supply_working/private_genset']['count']

In [59]:
percentage_ownership_series = (go_series/hh_series)*100
po_dict = {'households': hh_series, 'gensets': go_series, 'percentage': percentage_ownership_series}
po_df = pd.DataFrame(po_dict)
type(po_df)

pandas.core.frame.DataFrame

In [60]:
po_df

,gensets,households,percentage
Abar,23,38,60.526316
Ajau,12,14,85.714286
Asei,24,25,96.000000
Atamali,34,34,100.000000
Ayapo,112,113,99.115044
Babrongko,59,59,100.000000
Burawai,5,5,100.000000
Donday,42,43,97.674419
Ebunfauw,30,30,100.000000
Evale,21,22,95.454545


In [62]:
survey['access_type'] = pysentani.access_type(survey)



frequency = {'PLN_expenditure':'monthly',
'community_microgrid_expenditure':'monthly',
'genset_expenditure':'daily'}

multiplier = {'monthly':1, 'weekly':4, 'daily':30}

# create new columns with same monthly frequency
for column in frequency.keys():
    # adjust expenditure frequency
    expenditure = survey[column] * multiplier[frequency[column]]
    new_column = column + '_monthly'
    survey[new_column] = expenditure



pd.set_option('max_rows', 200)
survey.groupby(['access_type', 
                'village_name',
                'demand_point'])[['PLN_expenditure_monthly',
                                  'genset_expenditure_monthly', 
                                  'community_microgrid_expenditure_monthly']].agg(['mean','count'])

PLN_expenditure_monthly        \
                                                                 mean count   
access_type         village_name demand_point                                 
PLN_grid            Ajau         household               59166.666667    12   
                    Asei         church                           NaN     0   
                                 clinic                           NaN     0   
                                 community                        NaN     0   
                                 household              121875.000000    24   
                                 other                            NaN     0   
                    Babrongko    church                           NaN     0   
                                 household                        NaN     0   
                                 other                            NaN     0   
                                 shop                             NaN     0   
                    Burawai      household               60000.000000     5   
                                 other                  116666.666667     3   
                    Evale        household              114944.444444    18   
                    Flafow       clinic                 180000.000000     1   
                                 household              108823.529412    68   
                                 other                  100000.000000     1   
                                 school                 100000.000000     1   
                    Hobong       church                 250000.000000     1   
                                 household              107388.888889    18   
                    Khamayakha   church                 100000.000000     1   
                                 household              145454.545455    11   
                                 other                 4800000.000000     1   
                    Kheleubulow  household              239478.260870    23   
                                 other                  877413.793103    29   
                    Kwadeware    church                 200000.000000     1   
                                 household               90000.000000    28   
                                 other                            NaN     0   
                    Pantai_Yahim household               58076.923077    13   
                    Simporo      church                           NaN     0   
                                 community                        NaN     0   
                                 household                        NaN     0   
                                 other                            NaN     0   
                                 school                           NaN     0   
                                 shop                             NaN     0   
                    Sosiri       household               80638.888889    90   
                                 other                   25000.000000     1   
                                 school                  50000.000000     1   
                    Yakonde      church                  50000.000000     1   
                                 household               64905.660377    53   
                    Yobeh        church                 100000.000000     1   
                                 clinic                  24000.000000     1   
                                 household              138888.888889    27   
                                 other                            NaN     0   
                                 school                           NaN     0   
                    Yoka         household              162625.000000    16   
PLN_microgrid       Ayapo        church                           NaN     0   
                                 household                        NaN     0   
                                 other                            NaN     0   
                 